# Bonus Lab — Build Your Own AC Coupling Circuit
## From unipolar to bipolar: a capacitor, a resistor, and the truth about coupling

---

**Prerequisites:** Module 00, Lesson 02 (AC vs DC and Signals) — specifically the sections on coupling, unipolar vs bipolar signals, and Experiment 4 (multimeter readings).

**What you'll learn:**
- How to build an AC coupling circuit from two components
- Why a series capacitor blocks DC and passes AC
- Why the resistor to ground is essential (and what happens without it)
- The time constant (τ = RC) and how it controls settling time
- The cutoff frequency — AC coupling is really a high-pass filter
- How to convert the Fnirsi's unipolar output into a bipolar signal
- How to get correct AC voltage readings from the Klein MM300

**Equipment:** Fnirsi 2C53T oscilloscope + signal generator, Klein MM300 multimeter, breadboard, jumper wires

**Parts from your kit:**
- 1µF capacitor
- 100nF capacitor (for exploration)
- 10kΩ resistor
- 100kΩ resistor (for exploration)

**Estimated time:** 30–60 minutes

In [ ]:
# Setup — run this cell first
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sig
%matplotlib inline

plt.rcParams.update({
    'figure.figsize': (10, 5),
    'axes.grid': True,
    'font.size': 12,
    'lines.linewidth': 2,
    'grid.alpha': 0.3,
})
print("Setup complete.")

---
## 1. The Problem We're Solving

In Module 00, Experiment 4, we discovered something frustrating: the Klein MM300 multimeter gave incorrect AC voltage readings for the Fnirsi's output. The readings were too high because the Fnirsi generates **unipolar** signals (0V to 3V) and the multimeter expects **bipolar** signals (centered on 0V).

The oscilloscope's **AC coupling** mode fixed the *display* — switching to AC coupling removed the DC offset and centered the waveform on 0V. But that only changed what the scope showed. The signal itself was still unipolar.

What if we could build a circuit that actually *converts* the signal from unipolar to bipolar? Then we could feed the converted signal to the multimeter and get correct readings.

We can. The oscilloscope's AC coupling mode is just a capacitor and a resistor. We can build the same thing on a breadboard.

---
## 2. What AC Coupling Actually Is — The Full Picture

In Module 00 we defined **coupling** as how one stage connects to the next:

- **DC coupling** = a direct wire. Everything passes through — DC and AC alike.
- **AC coupling** = a series capacitor that blocks DC and passes AC.

But here's what we didn't cover: **the capacitor alone isn't the whole story.** A capacitor in series blocks DC from flowing *through* the connection, but the output side still needs to know what voltage to sit at when there's no signal. It needs a **DC reference** — a path to some known voltage.

Inside your oscilloscope, that reference is the **1MΩ input impedance** — a very high-value resistor from the input to ground that's always there. It pulls the DC level of the AC-coupled input to 0V.

On a breadboard, we add that resistor ourselves:

```
                    C              
Signal In ───┤├─────┬───── Signal Out (to scope / meter)
             1µF    │
                    R  10kΩ
                    │
                   GND
```

That's it. Two components.

- **C (series capacitor):** Blocks the DC component, passes the AC component.
- **R (resistor to ground):** Sets the DC operating point of the output to 0V. Without it, the output node would float at an undefined voltage.

Together, these two components form what's called a **high-pass filter** — it passes high frequencies (AC) and blocks low frequencies (DC is the ultimate low frequency: 0 Hz). We'll explore the filtering behavior later, but first let's understand *why* this works.

---
## 3. Why the Capacitor Blocks DC

A capacitor is two conductors separated by an insulator. No current can flow *through* the insulator — electrons can't cross the gap. But current can flow *in and out of* the capacitor as it charges and discharges.

When the voltage across a capacitor is **changing**, charge flows onto one plate and off the other — current flows in the external circuit. When the voltage is **constant** (DC), the capacitor is fully charged and current stops.

This is the key intuition:
- **DC (constant voltage):** Capacitor charges up, current stops. DC is blocked.
- **AC (changing voltage):** Capacitor is constantly charging and discharging. Current flows continuously. AC passes through.

### What happens with the Fnirsi's unipolar signal

The Fnirsi outputs a 0V-to-3V sine wave. This is really a **1.5V DC offset** with a **±1.5V AC component** riding on top:

$$v_{in}(t) = \underbrace{1.5\text{V}}_{\text{DC}} + \underbrace{1.5\text{V} \cdot \sin(2\pi f t)}_{\text{AC}}$$

When this hits the series capacitor:
1. The capacitor charges up to the **average (DC) value** of the input: 1.5V.
2. Once charged, only the **deviations from that average** — the AC component — appear on the output side.
3. The resistor to ground pulls the output's DC level to 0V.

Result: the output swings ±1.5V around 0V. **Bipolar.** Exactly what the multimeter expects.

In [ ]:
# --- Simulate the RC coupling circuit with startup transient ---

R = 10e3    # 10kΩ
C = 1e-6    # 1µF
tau = R * C  # time constant = 10ms
f = 1000    # 1 kHz signal
amplitude = 3.0  # Fnirsi amplitude setting (0V to 3V)

# Simulate for 80ms (8 time constants) to show settling
dt = 1e-6  # 1µs time step
t = np.arange(0, 0.08, dt)

# Input: unipolar sine wave (0V to 3V)
v_in = (amplitude / 2) + (amplitude / 2) * np.sin(2 * np.pi * f * t)

# Simulate RC circuit: v_out = v_in - v_cap, dv_cap/dt = (v_in - v_cap) / (R*C) * (R/(R+0))
# For a series C with R to ground: v_out appears across R
# v_cap charges through R: v_out = v_in - v_cap, i = v_out / R = C * dv_cap/dt
# But more precisely: v_out = R * i, and i = C * d(v_in - v_out)/dt... 
# Simplest: simulate with Euler method
v_cap = np.zeros_like(t)  # voltage across capacitor
v_out = np.zeros_like(t)  # voltage across R (output)

for i in range(1, len(t)):
    # Current through circuit: i = (v_in - v_cap) / R
    # (The output voltage v_out = i * R = v_in - v_cap)
    # Capacitor charges: dv_cap = i * dt / C = (v_in - v_cap) * dt / (R * C)
    i_current = (v_in[i-1] - v_cap[i-1]) / R
    v_cap[i] = v_cap[i-1] + i_current * dt / C
    v_out[i] = v_in[i] - v_cap[i]

# Plot
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

axes[0].plot(t * 1000, v_in, color='#2980b9')
axes[0].axhline(y=1.5, color='orange', linestyle='--', alpha=0.5, label='DC average (1.5V)')
axes[0].set_ylabel('Voltage (V)')
axes[0].set_title('Input: Fnirsi output (0V to 3V, unipolar)')
axes[0].set_ylim(-0.5, 3.5)
axes[0].legend()

axes[1].plot(t * 1000, v_cap, color='#e74c3c')
axes[1].axhline(y=1.5, color='orange', linestyle='--', alpha=0.5, label='Steady-state (1.5V)')
axes[1].set_ylabel('Voltage (V)')
axes[1].set_title(f'Capacitor voltage (charges to DC average over ~5τ = {5*tau*1000:.0f}ms)')
axes[1].set_ylim(-0.5, 3.5)
axes[1].legend()

axes[2].plot(t * 1000, v_out, color='#27ae60')
axes[2].axhline(y=0, color='black', linewidth=0.5)
axes[2].set_ylabel('Voltage (V)')
axes[2].set_xlabel('Time (ms)')
axes[2].set_title('Output: AC-coupled signal (±1.5V, bipolar)')
axes[2].set_ylim(-2.5, 3.5)

# Mark the settling region
for ax in axes:
    ax.axvspan(0, 5 * tau * 1000, alpha=0.08, color='red')
axes[2].text(5 * tau * 1000 / 2, 2.5, f'Settling\n(~5τ = {5*tau*1000:.0f}ms)',
             ha='center', fontsize=10, color='red', alpha=0.7)

plt.tight_layout()
plt.show()

print(f"τ = R × C = {R/1e3:.0f}kΩ × {C*1e6:.0f}µF = {tau*1000:.0f}ms")
print(f"After ~5τ ({5*tau*1000:.0f}ms), the capacitor is charged to ~1.5V")
print(f"and the output is a clean ±1.5V bipolar signal.")

---
## 4. The Time Constant

How long does the capacitor take to charge up? This is controlled by the **time constant**:

$$\tau = R \times C$$

For our circuit: $\tau = 10\text{k}\Omega \times 1\mu\text{F} = 10\text{ms}$

The capacitor charges exponentially:
- After **1τ** (10ms): 63% charged
- After **2τ** (20ms): 86% charged
- After **3τ** (30ms): 95% charged
- After **5τ** (50ms): 99.3% charged — effectively done

This is why you see a brief transient when you first connect the signal or switch the oscilloscope to AC coupling mode — the capacitor needs a few time constants to charge up to the DC average of the input.

### Choosing R and C

A larger time constant means slower settling but better low-frequency response. A smaller time constant means faster settling but the circuit starts attenuating lower frequencies. This tradeoff leads us to the next concept: the cutoff frequency.

In [ ]:
# --- Time constant for various R/C combinations from the shopping list ---

resistors = [1e3, 2.2e3, 4.7e3, 10e3, 47e3, 100e3]  # Ω
capacitors = [100e-9, 1e-6, 10e-6]  # F

def format_r(r):
    if r >= 1e3:
        return f"{r/1e3:.1f}kΩ" if r % 1e3 else f"{r/1e3:.0f}kΩ"
    return f"{r:.0f}Ω"

def format_c(c):
    if c >= 1e-6:
        return f"{c*1e6:.0f}µF"
    elif c >= 1e-9:
        return f"{c*1e9:.0f}nF"
    return f"{c*1e12:.0f}pF"

def format_tau(tau):
    if tau >= 1:
        return f"{tau:.1f}s"
    elif tau >= 1e-3:
        return f"{tau*1e3:.1f}ms"
    return f"{tau*1e6:.1f}µs"

def format_freq(f):
    if f >= 1e3:
        return f"{f/1e3:.1f}kHz"
    return f"{f:.1f}Hz"

print(f"{'R':<10} {'C':<10} {'τ = RC':<12} {'f_c = 1/(2πRC)':<15} {'5τ (settle)':<12}")
print("-" * 60)

for c in capacitors:
    for r in resistors:
        tau = r * c
        fc = 1 / (2 * np.pi * tau)
        marker = "  ◄── our circuit" if (r == 10e3 and c == 1e-6) else ""
        print(f"{format_r(r):<10} {format_c(c):<10} {format_tau(tau):<12} {format_freq(fc):<15} {format_tau(5*tau):<12}{marker}")
    print()

---
## 5. The Cutoff Frequency — AC Coupling Is a High-Pass Filter

This circuit doesn't just "block DC" — it's a **high-pass filter** with a specific cutoff frequency:

$$f_c = \frac{1}{2\pi R C}$$

For our circuit: $f_c = \frac{1}{2\pi \times 10\text{k}\Omega \times 1\mu\text{F}} \approx 15.9\text{ Hz}$

What this means:
- **Signals well above $f_c$** pass through with nearly full amplitude. Our 1 kHz test signal is ~63× above the cutoff — it passes with negligible loss.
- **At $f_c$** the signal is attenuated to 70.7% of its input amplitude (down 3 dB).
- **Signals well below $f_c$** are progressively blocked. At 1 Hz, almost nothing gets through.
- **DC (0 Hz)** is completely blocked.

The "3 dB" point at $f_c$ is the standard way to define the boundary of a filter. You'll see this on datasheets for amplifiers, filters, and communication systems throughout the course.

This is why choosing R and C matters: too small a product and your filter starts eating into the frequencies you want to keep.

In [ ]:
# --- Frequency response (Bode plot) of the high-pass RC filter ---

f = np.logspace(-1, 5, 1000)  # 0.1 Hz to 100 kHz

configs = [
    (10e3, 1e-6,   '#2980b9', '10kΩ + 1µF (our circuit)'),
    (10e3, 100e-9, '#e74c3c', '10kΩ + 100nF'),
    (100e3, 1e-6,  '#27ae60', '100kΩ + 1µF'),
]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

for R, C, color, label in configs:
    fc = 1 / (2 * np.pi * R * C)
    # Transfer function magnitude: |H(f)| = f / sqrt(f^2 + fc^2)
    H = f / np.sqrt(f**2 + fc**2)
    H_dB = 20 * np.log10(H)
    
    ax1.semilogx(f, H, color=color, linewidth=2, label=f'{label} (f_c = {format_freq(fc)})')
    ax2.semilogx(f, H_dB, color=color, linewidth=2, label=f'{label}')
    
    # Mark cutoff
    ax1.plot(fc, 1/np.sqrt(2), 'o', color=color, markersize=8)
    ax2.plot(fc, -3, 'o', color=color, markersize=8)

# Mark our signal frequency
ax1.axvline(x=1000, color='gray', linestyle=':', alpha=0.5)
ax1.text(1100, 0.5, '1 kHz\n(our signal)', fontsize=10, color='gray')
ax2.axvline(x=1000, color='gray', linestyle=':', alpha=0.5)

ax1.axhline(y=1/np.sqrt(2), color='orange', linestyle='--', alpha=0.3, label='-3dB (70.7%)')
ax1.set_ylabel('Gain (V_out / V_in)')
ax1.set_title('High-Pass Filter Frequency Response')
ax1.set_ylim(0, 1.1)
ax1.legend(fontsize=9)

ax2.axhline(y=-3, color='orange', linestyle='--', alpha=0.3)
ax2.set_ylabel('Gain (dB)')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylim(-40, 3)
ax2.legend(fontsize=9)

plt.tight_layout()
plt.show()

# Show attenuation at specific frequencies for our circuit
R, C = 10e3, 1e-6
fc = 1 / (2 * np.pi * R * C)
print(f"\nOur circuit: R = 10kΩ, C = 1µF, f_c = {fc:.1f} Hz")
print(f"{'Frequency':<15} {'Gain':>8} {'Output for 1.5V AC input':>25}")
print("-" * 50)
for freq in [1, 5, fc, 50, 100, 1000, 10000]:
    gain = freq / np.sqrt(freq**2 + fc**2)
    v_out = gain * 1.5
    label = format_freq(freq)
    if abs(freq - fc) < 0.1:
        label += " (f_c)"
    print(f"{label:<15} {gain:>7.3f}  {v_out:>7.3f}V peak")

---
## 6. Simulation — Build It in Falstad

[Falstad Circuit Simulator](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA) lets you build this circuit and watch the capacitor charge in real time.

### What to build

1. Open [Falstad (blank canvas)](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Place an **AC Voltage Source** (right-click → Draw → Inputs and Sources → Add AC Voltage). Set it to 1000 Hz, Max Voltage 1.5V, and add a **DC Offset of 1.5V** (this simulates the Fnirsi's 0-to-3V output).
3. Place a **Capacitor** in series with the source output. Set it to 1µF.
4. Place a **Resistor** from the capacitor's output to ground. Set it to 10kΩ.
5. Connect ground back to the source's negative terminal to complete the loop.

### What to observe

- Right-click the **AC source** → View in New Scope: you'll see the 0V-to-3V unipolar input.
- Right-click the **resistor** → View in New Scope: you'll see the ±1.5V bipolar output.
- Watch the startup transient — the output starts large and settles over a few time constants.
- Right-click the **capacitor** → View in New Scope: you'll see it charge up to ~1.5V.

### What to try

- Change the frequency to 10 Hz (near the cutoff) — the output amplitude drops.
- Change the frequency to 1 Hz — almost no signal gets through.
- Change R to 100kΩ — the cutoff drops, and now low frequencies pass better (but settling is slower).
- Remove R entirely — watch the output drift as the node floats.

---
## 7. Experiments

### Experiment 1: Build the AC Coupling Circuit

**Goal:** Build the circuit on a breadboard and observe the unipolar-to-bipolar conversion on the oscilloscope.

**Breadboard layout:**
```
     Fnirsi Signal Out (BNC)
           │
           │  ┌─────────────┐
     ──────┤├─┤ Node A      ├──── Scope Probe Tip (CH1)
        C=1µF │             │
              │  R = 10kΩ   │
              │     │       │
              │    GND      │
              └─────────────┘
                   │
              Scope Probe GND clip ──── Breadboard GND rail
              Fnirsi Signal GND    ──── Breadboard GND rail
```

**Important:** The Fnirsi signal ground and the oscilloscope probe ground must be connected to the same ground rail on the breadboard.

**Steps:**

1. Insert the **1µF capacitor** across the breadboard's center channel. One lead connects to the signal input, the other to Node A.
2. Insert a **10kΩ resistor** from Node A to the ground rail.
3. Connect the **Fnirsi signal output** to the capacitor's input side (and its ground to the ground rail).
4. Connect the **oscilloscope probe** to Node A (tip) and the ground rail (ground clip).
5. Set the Fnirsi to output a **1 kHz sine wave, Amplitude: 3V**.
6. Start signal generation (long-press ⏸️▶️ until the icon turns green).
7. Set the scope to **DC coupling**, **500µs/div**, **1V/div**.

**What you should see:**
- A sine wave centered on **0V**, swinging from **-1.5V to +1.5V**.
- V_pp = 3V — the same as the input, just shifted down.
- If you restart the signal generator, you may briefly see the waveform offset before settling (the startup transient). It settles in about 50ms (5τ), which may be too fast to catch by eye.

**If you have a second probe/channel:** Connect CH1 directly to the Fnirsi output and CH2 to Node A. You'll see the unipolar input and bipolar output simultaneously.

---
### Experiment 2: Fix the Multimeter — The Payoff

**Goal:** Connect the Klein MM300 to the AC-coupled output and verify that it now reads correctly.

**Setup:**
```
Fnirsi Signal Out ──┤├──┬──── Multimeter (AC V mode)
                  1µF   │
                       10kΩ
                        │
                       GND ──── Multimeter COM
```

**Steps:**

1. Keep the circuit from Experiment 1.
2. Connect the MM300 in **AC V** mode across Node A and ground (where the scope probe was).
3. Generate a **1 kHz sine wave at 3V amplitude**.
4. Read the multimeter.

**Expected reading:** The AC component is ±1.5V. For a sine wave:

$$V_{rms} = \frac{1.5\text{V}}{\sqrt{2}} \approx 1.06\text{V}$$

The MM300 should now read approximately **1.06V** — matching the theoretical value. Compare this with whatever you measured in Module 00 Experiment 4 without the coupling circuit.

5. Switch to a **1 kHz square wave at 3V amplitude**.

The MM300 is still an average-responding meter, so it will still misread the square wave — but the error will be the *expected* average-responding error (reading ~1.5V × 1.11 ≈ 1.67V instead of the true RMS of 1.5V), not the anomalous readings from Experiment 4. The DC contamination problem is gone.

> **What just happened:** You solved the measurement problem from Module 00 with two passive components. The signal is now bipolar, the meter's assumptions are satisfied, and the reading is correct. This is exactly what happens inside every AC-coupled instrument.

In [ ]:
# --- Expected readings: with and without coupling circuit ---

amplitude = 3.0  # Fnirsi amplitude setting
V_dc = amplitude / 2
V_ac_peak = amplitude / 2

# True RMS values of the AC component
V_rms_sine = V_ac_peak / np.sqrt(2)
V_rms_square = V_ac_peak  # square wave RMS = peak

# Average-responding meter calibrated for sine:
# It measures V_avg = (2/π) * V_peak, then multiplies by π/(2√2) ≈ 1.1107
# For sine: this gives correct RMS
# For square: V_avg of a square wave = V_peak, so meter reads V_peak * 1.1107
sine_factor = 1.1107  # π / (2√2)

print("Expected Readings: AC-Coupled Output")
print("=" * 65)
print(f"Fnirsi amplitude: {amplitude}V → AC component: ±{V_ac_peak}V")
print()
print(f"{'Waveform':<12} {'True RMS':>10} {'MM300 (coupled)':>16} {'Correct?':>10}")
print("-" * 50)
print(f"{'Sine':<12} {V_rms_sine:>8.3f} V {V_rms_sine:>14.3f} V {'Yes':>10}")
print(f"{'Square':<12} {V_rms_square:>8.3f} V {V_ac_peak * sine_factor:>14.3f} V {'No*':>10}")
print()
print("* The square wave reading is wrong because the MM300 is average-responding,")
print("  not true-RMS. But this is the EXPECTED error for a non-true-RMS meter —")
print("  it's no longer contaminated by DC.")
print()
print("Compare with your Module 00 Experiment 4 readings (without coupling):")
print("  If those readings were significantly higher than the values above,")
print("  the difference was caused by DC contamination — now eliminated.")

---
### Experiment 3: Explore the Filter's Limits

**Goal:** See that this circuit is genuinely a frequency-dependent filter, not just a "DC blocker."

Our cutoff frequency is $f_c \approx 15.9\text{ Hz}$. Signals far above this pass through; signals near or below it get attenuated.

**Steps:**

1. Keep the 10kΩ + 1µF circuit. Connect the oscilloscope to Node A (**DC coupling** so you can see the true output level).

2. **Sweep the frequency** and record V_pp at the output for each:

| Frequency | Expected V_pp (out of 3V input) | What you should see |
|-----------|-------------------------------|--------------------|
| 1 kHz | ~3.0V (99.99% passes) | Full-size sine wave centered on 0V |
| 100 Hz | ~2.99V (99.7% passes) | Nearly identical to 1 kHz |
| 50 Hz | ~2.95V (98.4% passes) | Very slight reduction |
| 15 Hz | ~2.12V (70.7% — this is f_c!) | Noticeably smaller |
| 5 Hz | ~0.93V (31.0%) | Much smaller |
| 1 Hz | ~0.19V (6.3%) | Almost gone |

3. **Try DC:** Disconnect the Fnirsi. Connect the bench supply (set to 5V) through the capacitor. The output should read 0V (after a brief transient). DC is completely blocked.

4. **Change components** to shift the cutoff:
   - Swap in **100kΩ** (keep 1µF): $f_c$ drops to 1.59 Hz. Now 5 Hz passes with 95% amplitude.
   - Swap in **100nF** (keep 10kΩ): $f_c$ rises to 159 Hz. Now 100 Hz is at the cutoff, and 50 Hz is attenuated to 30%.

This is the tradeoff: a lower cutoff frequency means better low-frequency response but slower settling (larger τ).

In [ ]:
# --- Visualize output amplitude vs frequency for different R/C values ---

f = np.logspace(-1, 4, 500)  # 0.1 Hz to 10 kHz
V_in_pp = 3.0  # Fnirsi output V_pp

configs = [
    (10e3, 1e-6,   '#2980b9', 'solid',  '10kΩ + 1µF'),
    (100e3, 1e-6,  '#27ae60', 'solid',  '100kΩ + 1µF'),
    (10e3, 100e-9, '#e74c3c', 'dashed', '10kΩ + 100nF'),
]

fig, ax = plt.subplots(figsize=(12, 6))

for R, C, color, ls, label in configs:
    fc = 1 / (2 * np.pi * R * C)
    gain = f / np.sqrt(f**2 + fc**2)
    V_out_pp = gain * V_in_pp
    ax.semilogx(f, V_out_pp, color=color, linestyle=ls, linewidth=2,
                label=f'{label} (f_c = {format_freq(fc)})')
    ax.plot(fc, V_in_pp / np.sqrt(2), 'o', color=color, markersize=8)

# Mark key frequencies
for freq, label in [(1, '1 Hz'), (15.9, 'f_c'), (1000, '1 kHz')]:
    ax.axvline(x=freq, color='gray', linestyle=':', alpha=0.3)

ax.axhline(y=V_in_pp, color='gray', linestyle='--', alpha=0.3, label=f'Input V_pp = {V_in_pp}V')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Output V_pp (V)')
ax.set_title('Output Amplitude vs Frequency — How the Filter Affects Your Signal')
ax.set_ylim(0, 3.5)
ax.legend(fontsize=10)

plt.tight_layout()
plt.show()

print("Dots mark the cutoff frequency (f_c) where V_pp drops to 70.7% of input.")
print("At 1 kHz, all three configurations pass the signal with negligible loss.")
print("The difference shows up at lower frequencies.")

---
## 8. Why the Resistor Matters

What happens if you remove R from the circuit? Just a capacitor in series, nothing else.

**The output node has no DC path to ground.** It's floating. In theory, the DC voltage at Node A is undefined — it could be anything. In practice, it drifts slowly based on tiny leakage currents through the capacitor's insulator, the oscilloscope's input, and even the air.

If the oscilloscope is connected, its **1MΩ input impedance** acts as R. But that's a very large R:

$$\tau = 1\text{M}\Omega \times 1\mu\text{F} = 1\text{ second}$$

So the circuit takes about **5 seconds** to settle instead of 50ms. You'll see the waveform slowly drift to its final position.

And if you connect a multimeter instead of a scope, the meter's input impedance is typically 10MΩ — giving τ = 10 seconds. The reading takes forever to stabilize.

The lesson: **every AC-coupled node needs an intentional DC bias path.** Don't rely on leakage or instrument input impedance.

---
### Experiment 4: Remove the Resistor

**Goal:** See what happens when the output node floats.

**Steps:**

1. **Remove the 10kΩ resistor** from the breadboard. Leave the capacitor in place. The scope probe is still on Node A.
2. Set the scope to **DC coupling** so you can see the actual DC level, not just AC.
3. Generate 1 kHz sine at 3V.
4. **Watch the waveform.** It should appear at some arbitrary DC level and slowly drift toward being centered on 0V — but the settling takes much longer than before (seconds instead of milliseconds, because the only R is the scope's 1MΩ input impedance).
5. Try restarting the signal generator — the waveform may jump to a different DC level and slowly settle again.
6. **Reconnect the 10kΩ resistor.** The waveform should snap to center on 0V within ~50ms.

This is why you can't just throw a capacitor in series and call it AC coupling. The resistor is doing real work.

In [ ]:
# --- Simulate: with R vs without R (using scope's 1MΩ as the only DC path) ---

f = 1000
amplitude = 3.0
C = 1e-6
dt = 1e-5

configs = [
    (10e3,  0.08, 'With 10kΩ resistor (τ = 10ms)'),
    (1e6,   8.0,  'Without resistor — scope 1MΩ only (τ = 1s)'),
]

fig, axes = plt.subplots(len(configs), 1, figsize=(12, 8))

for ax, (R, duration, title) in zip(axes, configs):
    tau = R * C
    t = np.arange(0, duration, dt)
    v_in = (amplitude / 2) + (amplitude / 2) * np.sin(2 * np.pi * f * t)
    
    v_cap = np.zeros_like(t)
    v_out = np.zeros_like(t)
    
    for i in range(1, len(t)):
        i_current = (v_in[i-1] - v_cap[i-1]) / R
        v_cap[i] = v_cap[i-1] + i_current * dt / C
        v_out[i] = v_in[i] - v_cap[i]
    
    # For long simulations, subsample for plotting
    step = max(1, len(t) // 50000)
    ax.plot(t[::step] * 1000, v_out[::step], color='#2980b9', linewidth=0.8)
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.set_ylabel('Output (V)')
    ax.set_title(f'{title} — settles in ~{5*tau*1000:.0f}ms' if tau < 1 
                 else f'{title} — settles in ~{5*tau:.0f}s')
    ax.set_ylim(-2, 3.5)
    
    # Mark 5τ
    settle_ms = 5 * tau * 1000
    if settle_ms < duration * 1000:
        ax.axvline(x=settle_ms, color='red', linestyle='--', alpha=0.5)
        ax.text(settle_ms, 3.0, f' 5τ = {settle_ms:.0f}ms' if settle_ms < 1000 
                else f' 5τ = {settle_ms/1000:.0f}s', color='red', fontsize=10)

axes[-1].set_xlabel('Time (ms)')
plt.tight_layout()
plt.show()

print("Top: With R = 10kΩ, the output settles to ±1.5V in ~50ms.")
print("Bottom: Without R, the scope's 1MΩ input is the only DC path.")
print("The output takes ~5 seconds to settle — and any disturbance resets it.")

---
## 9. Where AC Coupling Shows Up in Real Circuits

The circuit you just built isn't a special lab exercise — it's one of the most common building blocks in electronics:

**Audio amplifier stages.** Each amplifier stage has its own DC bias point (the voltages that keep the transistors in their operating range). A coupling capacitor between stages passes the audio signal while blocking one stage's DC bias from disturbing the next. Every guitar amp, PA system, and headphone amplifier has these.

**Your oscilloscope's AC coupling mode.** Literally this circuit: a capacitor in series with the input, and the scope's 1MΩ input impedance as R. Now you know why there's a brief settling time when you switch from DC to AC coupling.

**Audio jacks and cables.** Many audio outputs include a series capacitor to block DC. This prevents damage if the output is accidentally shorted and ensures that no DC offset reaches the headphones or speaker.

**High-speed digital links.** USB 3.0, PCIe, SATA, and Ethernet all use AC coupling at the physical layer. The same principle — series capacitor, bias resistors on the receiver — running at GHz frequencies with tiny capacitors (100nF or smaller).

**Datasheet connection.** When a datasheet specifies an "AC-coupled input" or shows a capacitor on the input in the application circuit, it's telling you that the device's internal circuitry sets its own DC bias point, and you just need to pass the AC signal through. The recommended capacitor value is chosen so that $f_c$ is well below the signal frequency — exactly the design exercise you worked through in this lab.

---
## Summary

| Concept | Key Point |
|---------|-----------|
| AC coupling circuit | Series capacitor + resistor to ground |
| Capacitor's role | Blocks DC (charges to average), passes AC |
| Resistor's role | Sets DC operating point of output — essential, not optional |
| Time constant | τ = RC — determines settling time (5τ to fully settle) |
| Cutoff frequency | $f_c = 1/(2\pi RC)$ — signals below this are attenuated |
| High-pass filter | AC coupling IS a high-pass filter |
| Practical result | Converts Fnirsi's unipolar output to bipolar → correct MM300 readings |
| Design tradeoff | Large RC: better low-freq response, slower settling. Small RC: faster settling, worse low-freq response. |

**What you built is exactly what's inside your oscilloscope's AC coupling mode.** The only difference is the value of R — the scope uses its 1MΩ input impedance, while we used 10kΩ for faster settling. Both block DC and pass AC.

---
## Checkpoint Questions

**Q1:** You have a 100nF capacitor and a 47kΩ resistor. What is τ? What is $f_c$?

**Q2:** You need to AC-couple a 20 Hz audio signal (the lowest note on a bass guitar) without significant attenuation. If R = 10kΩ, what's the minimum capacitor value? (Hint: you want $f_c$ to be well below 20 Hz — aim for $f_c \leq$ 2 Hz.)

**Q3:** What happens if you make C too small for your signal frequency? What would you see on the oscilloscope?

**Q4:** When you switch your oscilloscope from DC coupling to AC coupling, you sometimes see the waveform jump and then slowly settle back. Explain why, using what you learned in this lab.

In [ ]:
# --- Checkpoint Answers (run to reveal) ---

print("=" * 60)
print("CHECKPOINT ANSWERS")
print("=" * 60)

# Q1
R_q1, C_q1 = 47e3, 100e-9
tau_q1 = R_q1 * C_q1
fc_q1 = 1 / (2 * np.pi * tau_q1)
print(f"\nQ1: τ = R × C = 47kΩ × 100nF = {tau_q1*1000:.1f}ms")
print(f"    f_c = 1/(2πτ) = 1/(2π × {tau_q1*1000:.1f}ms) = {fc_q1:.1f} Hz")

# Q2
fc_target = 2.0  # Hz
R_q2 = 10e3
C_q2 = 1 / (2 * np.pi * R_q2 * fc_target)
print(f"\nQ2: For f_c ≤ 2 Hz with R = 10kΩ:")
print(f"    C ≥ 1/(2π × R × f_c) = 1/(2π × 10kΩ × 2Hz)")
print(f"    C ≥ {C_q2*1e6:.1f}µF")
print(f"    Use a 10µF capacitor from your kit (gives f_c = {1/(2*np.pi*10e3*10e-6):.2f} Hz).")

# Q3
print(f"\nQ3: If C is too small, f_c is too high, and your signal frequency")
print(f"    falls near or below the cutoff. The output amplitude shrinks —")
print(f"    you see a smaller waveform than the input. At frequencies well")
print(f"    below f_c, the signal nearly disappears.")
print(f"    Example: 100pF + 10kΩ → f_c = {1/(2*np.pi*10e3*100e-12)/1e3:.0f} kHz.")
print(f"    A 1 kHz signal would be severely attenuated.")

# Q4
print(f"\nQ4: When you switch to AC coupling, the scope inserts a capacitor")
print(f"    in series with the input. That capacitor starts uncharged, so the")
print(f"    output initially includes the full DC level of the signal. The")
print(f"    capacitor then charges up to the DC average through the scope's")
print(f"    1MΩ input impedance (τ = C × 1MΩ). After several time constants,")
print(f"    the DC is blocked and the waveform settles centered on 0V.")
print(f"    This is exactly the startup transient you saw in Experiments 1 & 4.")